# PyTao plotting with Matplotlib

PyTao supports plotting directly from the notebook, without a separate (X11) plot window.

PyTao provides two backends:
* Bokeh (with interactive plotting support)
* Matplotlib 

When plotting is enabled, PyTao will automatically select the best available backend.


The plotting backend may be specified explicitly, as we will do in this notebook in order to show off
this backend's functionality.

---
## Tao setup

In [ ]:
%config InlineBackend.figure_format = "retina"

In [ ]:
import pytao
from pytao import Tao, SubprocessTao

import matplotlib.pyplot as plt

In [ ]:
init_file = "$ACC_ROOT_DIR/bmad-doc/tao_examples/optics_matching/tao.init"

tao = Tao(init_file=init_file, plot="mpl")

## Shape setup

Let's update Tao's plotting shapes first, so that we see label names in the layout and floor plan.
Customizing this in the Bmad init files is an alternative to doing this in PyTao.

In [ ]:
tao.update_plot_shapes("quadrupole", type_label="name", layout=True, floor=True);

## The floor plan

In [ ]:
tao.plot("floor_plan", ylim=(-2, 2), figsize=(6, 4))

## Single data plots

In [ ]:
tao.plot("dispersion", include_layout=True)

## Plot fields

In [ ]:
tao.cmd("set var quad[1]|model = -5")
tao.plot_field("Q1")

## Stacked plots

In [ ]:
tao.plot(["alpha", "beta"])

In [ ]:
tao.plot(["alpha", "beta"], include_layout=False)

## Gridded plots

In [ ]:
tao.plot(["beta", "dispersion", "orbit"], grid=(2, 2), figsize=(8, 8))

## Saving plots

The parameter `save` makes it convenient to simultaneously display and save the plot to a file.

In [ ]:
tao.plot("beta", save="beta.png", figsize=(3, 3), include_layout=False)

## Customized plots

In [ ]:
tao.plot(["beta", "dispersion", "orbit"], grid=(2, 2))

# Access the figure by using `plt.gcf()` ("get current figure")
fig = plt.gcf()
fig.suptitle("Customized plot title")

# Access individual Axes objects by indexing `fig.axes`:
fig.axes[0].set_title("Beta [model]")
fig.tight_layout()

## Change defaults

In [ ]:
tao.matplotlib.configure(layout_height=0.25, figsize=(4, 4))

In [ ]:
tao.plot("beta")

## Accessing plot data

In [ ]:
graphs, beta_fig, beta_ax = tao.last_plot

In [ ]:
graphs[0].curves[0].line.xs[:10]

## Advanced plotting settings

`TaoGraphSettings` may be used to customize per-graph settings in single or gridded plots.

Since each graph has its own settings, we specify a list of `TaoGraphSettings`.
`TaoGraphSettings` includes customization of titles, legends, scales, per-axis settings (with `TaoAxisSettings`), and so on.  

For example, to change the title of a plot, one could use:
`TaoGraphSettings(title="something")` - or equivalently a custom Tao command can be sent with `TaoGraphSettings(commands=["set graph {graph} title = something"])`.

See `TaoGraphSettings` documentation for further information on what may be customized. Not all settings will be supported by PyTao's plotting backends.

In [ ]:
from pytao.plotting import TaoGraphSettings, TaoAxisSettings

# Let's use SubprocessTao to make an independent Tao instance in a subprocess.
# Now we can use `tao` from above (`optics_matching`) and `erl` here simultaneously.
erl = SubprocessTao(
    init_file="$ACC_ROOT_DIR/bmad-doc/tao_examples/erl/tao.init", 
    plot="mpl", 
    # The ERL example startup file customizes plots in a way incompatible with pytao's external plotting.
    # Set "nostartup=True" to avoid this.
    nostartup=True,
)

erl.cmds(
    [
        "set global track_type = beam",
        "set var r56[1]|model = 0.234",
        "set var t566[1]|model = 0.567",
    ]
)

erl.plot(
    "alpha",
    settings=TaoGraphSettings(
        title="My Custom Alpha Plot",
        component="model",
        draw_grid=False,
        x=TaoAxisSettings(
            label="Position - s [m]",
        ),
    ),
)

## Advanced curve settings

`TaoCurveSettings` may be used to customize per-curve settings in simple or gridded plots.

The below example has 4 plots in a 2x2 grid.

Since each plot has a set of curves, we must specify a dictionary for each plot.

That dictionary contains a mapping of `curve_index` (starting with 1) to a `TaoCurveSettings` instance.

See `TaoCurveSettings` for further information on what may be customized.

In [ ]:
from pytao.plotting import TaoCurveSettings

erl.plot(
    ["zphase", "zphase", "zphase", "zphase2"],
    grid=(2, 2),
    curves=[
        {1: TaoCurveSettings(ele_ref_name=r"linac.beg\1")},
        {1: TaoCurveSettings(ele_ref_name=r"linac.end\1")},
        {1: TaoCurveSettings(ele_ref_name=r"linac.beg\2")},
        {1: TaoCurveSettings(ele_ref_name=r"linac.end\2")},
    ],
    xlim=(-3, 3),
    ylim=(-10, 10),
    share_x=False,
    include_layout=False,
    figsize=(6, 6),
)